In [47]:
import os
import pandas as pd
import numpy as np
import json
import sys

In [48]:
# Get the current working directory
CURRENT_DIRECTORY = os.getcwd()
# Get the parent directory
PARENT_DIRECTORY = os.path.dirname(CURRENT_DIRECTORY)
PARENT_DIRECTORY = os.path.dirname(PARENT_DIRECTORY)

sys.path.append(PARENT_DIRECTORY + '//config/')
from config import count_sequences_above_threshold, count_sequences_below_threshold, normalize_sensor_data

# Open the config file and load its content into a dictionary
config_file = open(PARENT_DIRECTORY + '\\config\\config.json')
CONFIG_DATA = json.load(config_file)

# Close the file after loading the data
config_file.close()

In [49]:
NORMALIZE_SENSOR_DATA = CONFIG_DATA['NORMALIZE_SENSOR_DATA']
WINDOW_SIZE = CONFIG_DATA['WINDOW_SIZE']
NUMBER_OF_FEATURES = 8

ROTATION_THRESHOLD = CONFIG_DATA['STEPS_PITCH_ROTATION_THRESHOLD'] if NORMALIZE_SENSOR_DATA == False else normalize_sensor_data(CONFIG_DATA['STEPS_PITCH_ROTATION_THRESHOLD'])

CLASSIFICATIONS = CONFIG_DATA['CLASSES_MOTIONSPEED']

LABEL_COLUMN = 'Class_MotionSpeed'

# processed data folder path
DATA_FOLDER = PARENT_DIRECTORY + '\\processed-training-data\\4-PROCESSED-DATA\TRAIN2\\'

# save numpy data folder
NUMPY_DATA_FOLDER_FILE_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\'

# v0 data (not normalized, use to generate more data)
ALL_X_TRAIN_CLASSIFY_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\training\\ALL-X-TRAIN-CLASSIFY-MOTIONSPEED-LAR-STEPS-V0.npy'
ALL_Y_TRAIN_CLASSIFY_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\training\\ALL-Y-TRAIN-CLASSIFY-MOTIONSPEED-LAR-STEPS-V0.npy'

In [50]:
LABEL_TO_CATEGORY = {label: category for category, label in enumerate(CLASSIFICATIONS)}

print(LABEL_TO_CATEGORY)

{'SLOW': 0, 'AVERAGE': 1, 'FAST': 2}


In [51]:
file_names = [file for file in os.listdir(DATA_FOLDER) if file.endswith('.xlsx') and "STEPS-LR-LAR" in file and os.path.isfile(os.path.join(DATA_FOLDER, file))]

print(file_names)

['PROC-TRAIN2-STEPS-LR-LAR-100BPM.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-110BPM.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-112BPM.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-120BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-124BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-160BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-164BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-196BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-200BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-220BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-224BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-30BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-31BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-40BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-41BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-49BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-50BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-55BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-56BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-60BPM.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-62BPM.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-80BPM.xlsx', 'P

In [55]:
def df2Xy(df, windowSize=5):
  X = []
  y = []

  # NORMALIZE THE SENSOR DATA
  if NORMALIZE_SENSOR_DATA == True:
    df["L_Pitch"] = df["L_Pitch"].apply(normalize_sensor_data)
    df["L_Roll"] = df["L_Roll"].apply(normalize_sensor_data)
    df["R_Pitch"] = df["R_Pitch"].apply(normalize_sensor_data)
    df["R_Roll"] = df["R_Roll"].apply(normalize_sensor_data)
    df["L_Pitch_Delta"] = df["L_Pitch_Delta"].apply(normalize_sensor_data)
    df["L_Roll_Delta"] = df["L_Roll_Delta"].apply(normalize_sensor_data)
    df["R_Pitch_Delta"] = df["R_Pitch_Delta"].apply(normalize_sensor_data)
    df["R_Roll_Delta"] = df["R_Roll_Delta"].apply(normalize_sensor_data)

  for i in range( len(df) - windowSize + 1):
    # inputs: X rows
    # form a new input which has size of our windowSize
    input_data_list_deltas = []
    input_data_list_readings = []

    # loop through each row in our windowsize
    for j in range(windowSize):
        # fetch sensor data for this row
        row_values_deltas = df.loc[i + j, ['L_Pitch_Delta', 'L_Roll_Delta', 'R_Pitch_Delta', 'R_Roll_Delta']].values.tolist()
        row_values_readings = df.loc[i + j, ['L_Pitch', 'L_Roll', 'R_Pitch', 'R_Roll']].values.tolist()

        # add row values to the input
        input_data_list_deltas.append(row_values_deltas)
        input_data_list_readings.append(row_values_readings)

    # turnn list into array to do arthimetic
    raw_sensor_data = np.array(input_data_list_readings)
    delta_sensor_data = np.array(input_data_list_deltas)

    # remove all negative sinces we want to the total change (we dont care which direction)
    input_calculate_deltas = np.abs(delta_sensor_data)

    # calculate the total change for each sennsor value (TOTAL POSITIVE SENSOR CHANGES)
    input_calculate_deltas = np.sum(input_calculate_deltas, axis=0)

    # calculate total sequences
    input_total_sequences = count_sequences_above_threshold(raw_sensor_data, ROTATION_THRESHOLD)

    input_data_array = np.concatenate((input_calculate_deltas, input_total_sequences))

    if(np.isnan(input_data_array).any() == False):
      # turn back to list
      input_data_list = input_data_array.tolist()

      # add our input to our total inputs, marked as X
      X.append(input_data_list)

      # outputs: y labels
      label = df.loc[i + (windowSize - 1), [LABEL_COLUMN]].values.tolist()

      y.append(label)

  return (np.array(X), np.array(y))

In [56]:
ALL_X_TRAIN = np.empty((0, NUMBER_OF_FEATURES))  # List to store all X training data
ALL_Y_TRAIN = np.empty((0))  # List to store all Y training data

In [57]:
def processData(ALL_X, ALL_Y):
    for fileName in file_names:
        # Read the Excel file
        df = pd.read_excel(DATA_FOLDER + fileName)

        xTrain, yTrain = df2Xy(df, WINDOW_SIZE)

        yTrain = yTrain.reshape(-1)
        
        ALL_X = np.concatenate([ALL_X, xTrain])
        ALL_Y = np.concatenate([ALL_Y, yTrain])
        
        print('xTrain.shape:', xTrain.shape)
        print('ALL_X_TRAIN.shape:', ALL_X.shape, ' ALL_Y_TRAIN.shape:', ALL_Y.shape)
        print('-------------------------------')

    # Create the folder if it doesn't exist
    if not os.path.exists(NUMPY_DATA_FOLDER_FILE_PATH):
        os.makedirs(NUMPY_DATA_FOLDER_FILE_PATH)

    # convert to numerical labels (originally text labels)
    numerical_label = np.vectorize(LABEL_TO_CATEGORY.get)(ALL_Y)

    # Assuming your numpy array is called 'data_array'
    np.save(ALL_X_TRAIN_CLASSIFY_PATH, ALL_X)
    np.save(ALL_Y_TRAIN_CLASSIFY_PATH, numerical_label)

In [58]:
processData(ALL_X_TRAIN, ALL_Y_TRAIN)

ALL_X_TRAIN_CLASSIFY = np.load(ALL_X_TRAIN_CLASSIFY_PATH)
ALL_Y_TRAIN_CLASSIFY = np.load(ALL_Y_TRAIN_CLASSIFY_PATH)

print(ALL_X_TRAIN_CLASSIFY.shape)
print(ALL_Y_TRAIN_CLASSIFY.shape)

xTrain.shape: (3381, 8)
ALL_X_TRAIN.shape: (3381, 8)  ALL_Y_TRAIN.shape: (3381,)
-------------------------------
xTrain.shape: (3274, 8)
ALL_X_TRAIN.shape: (6655, 8)  ALL_Y_TRAIN.shape: (6655,)
-------------------------------
xTrain.shape: (4560, 8)
ALL_X_TRAIN.shape: (11215, 8)  ALL_Y_TRAIN.shape: (11215,)
-------------------------------
xTrain.shape: (3772, 8)
ALL_X_TRAIN.shape: (14987, 8)  ALL_Y_TRAIN.shape: (14987,)
-------------------------------
xTrain.shape: (3180, 8)
ALL_X_TRAIN.shape: (18167, 8)  ALL_Y_TRAIN.shape: (18167,)
-------------------------------
xTrain.shape: (3071, 8)
ALL_X_TRAIN.shape: (21238, 8)  ALL_Y_TRAIN.shape: (21238,)
-------------------------------
xTrain.shape: (4504, 8)
ALL_X_TRAIN.shape: (25742, 8)  ALL_Y_TRAIN.shape: (25742,)
-------------------------------
xTrain.shape: (3795, 8)
ALL_X_TRAIN.shape: (29537, 8)  ALL_Y_TRAIN.shape: (29537,)
-------------------------------
xTrain.shape: (3380, 8)
ALL_X_TRAIN.shape: (32917, 8)  ALL_Y_TRAIN.shape: (32917,)
-

In [59]:
print(np.max(ALL_X_TRAIN_CLASSIFY))

227.0
